# AirSeaFlux: bulkformulae Optimization Problem

The notebook `optim_and_enzyme.ipynb` includes several examples the use Enzyme and Optim to get adjoints and to minimize some cost function. The input in most cases is a vector, but the output of the function we are optimizing is scalar (a single term from the bulkformulae function). Here we are working to build a version that uses all outputs of bulkformulae.

In [9]:
using Pkg; Pkg.add(url="https://github.com/eldavenport/ECCO.jl"); Pkg.add("AirSeaFluxes")
using ECCO
import AirSeaFluxes: bulkformulae
Pkg.add("Enzyme")
using Enzyme, Optim
Pkg.add("LinearAlgebra")
using LinearAlgebra

    Updating git-repo `https://github.com/eldavenport/ECCO.jl`
   Resolving package versions...
  No Changes to `/opt/julia-1.10.5/share/julia/environments/v1.10/Project.toml`
  No Changes to `/opt/julia-1.10.5/share/julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/julia-1.10.5/share/julia/environments/v1.10/Project.toml`
  No Changes to `/opt/julia-1.10.5/share/julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/julia-1.10.5/share/julia/environments/v1.10/Project.toml`
  No Changes to `/opt/julia-1.10.5/share/julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
    Updating `/opt/julia-1.10.5/share/julia/environments/v1.10/Project.toml`
  [37e2e46d] + LinearAlgebra
  No Changes to `/opt/julia-1.10.5/share/julia/environments/v1.10/Manifest.toml`


In [13]:
x0 = [300.0,0.001,1.0,10.0]

function J_bulkformulae(x::Vector{Float64})
    obs = [-3.1,2.1,5.5e-9,0.05]
    res = bulkformulae(x[1],x[2],x[3],x[4])
    y = [res[1],res[2],res[3],res[6]]
    
    J = norm(y-obs)^2
    return J
end

# J_bulkformulae(x0)

function J_ad!(dx2, x) 
    dx = zeros(size(x))
    Enzyme.autodiff(Reverse, J_bulkformulae, Duplicated(x, dx))
    dx2 .= dx
end

# for testing: evaluate the gradient at x0
dx2 = zeros(size(x0))
J_ad!(dx2,x0)

# optimization with the cost function and it's adjoint 
result=Optim.optimize(J_bulkformulae, J_ad!, x0, Optim.Options(show_trace=true))
x1=Optim.minimizer(result)

# check that tau at x1 is close to y_obs
y1 = bulkformulae(x1[1],x1[2],x1[3],x1[4])
(hl=y1.hl,hs=y1.hs,evap=y1.evap,tau=y1.tau)

Iter     Function value   Gradient norm 
     0     8.369224e-03     3.615157e+01
 * time: 7.486343383789062e-5
     1     7.535613e-03     2.459393e+01
 * time: 0.00029087066650390625
     2     6.817075e-03     2.917685e-01
 * time: 0.00043082237243652344
     3     1.650358e-03     2.663119e+00
 * time: 0.0005710124969482422
     4     1.642103e-03     2.666215e-02
 * time: 0.0006718635559082031
     5     1.642075e-03     6.924692e-05
 * time: 0.0007739067077636719
     6     1.642075e-03     2.035970e-03
 * time: 0.000885009765625
     7     1.628204e-03     1.349200e+00
 * time: 0.0010190010070800781
     8     1.622217e-03     2.273158e+00
 * time: 0.0011069774627685547
     9     1.591295e-03     7.702166e-01
 * time: 0.0012128353118896484
    10     1.589195e-03     1.073986e+00
 * time: 0.0013108253479003906
    11     1.583926e-03     1.247671e+00
 * time: 0.0014109611511230469
    12     1.579176e-03     1.426437e+00
 * time: 0.0015099048614501953
    13     1.572024e-03   

┌ Warning: Using fallback BLAS replacements for (["dasum_64_"]), performance may be degraded
└ @ Enzyme.Compiler ~/.julia/packages/GPUCompiler/QCNA0/src/utils.jl:59


(hl = -3.100000000000336, hs = 2.099999999999938, evap = 1.2402480496100564e-9, tau = 0.05000000000005694)